# Scope del Proyecto

## Objetivo general:

* Analizar la satisfacción de los pasajeros de las aerolíneas y determinar los factores que tienen el mayor impacto en su satisfacción.

## Objetivos específicos:

* Identificar las variables que están más correlacionadas con la satisfacción de los pasajeros.
* Analizar la relación entre la satisfacción de los pasajeros y los diferentes aspectos del servicio de la aerolínea, como la comodidad del asiento, el entretenimiento a bordo, la limpieza, etc.
* Determinar si hay diferencias significativas en la satisfacción de los pasajeros según la aerolínea, el tipo de viaje (nacional/internacional), la clase de viaje, el destino, etc.
* Identificar patrones en la satisfacción de los pasajeros a lo largo del tiempo.

## Preguntas Planteadas
1. ¿Cuantos clientes leales tuvieron vuelos satisfactorios?
2. ¿A qué sexo (masculino y femenino) le costó más agendar un vuelo en línea?
3. ¿Cuáles son los factores que tienen el mayor impacto en la satisfacción de los pasajeros?
4. ¿Hay alguna diferencia en la satisfacción de los pasajeros entre las diferentes aerolíneas?
5. ¿La satisfacción de los pasajeros difiere según el tipo de viaje o la clase de viaje?
6. ¿Existe algún patrón en la satisfacción de los pasajeros a lo largo del tiempo?

# Exploración de datos

In [2]:
# Importación de librerias utilizadas en el proyecto
import numpy as np
import pandas as pd

In [7]:
# Se cargan los datos del archivo CSV en un DataFrame de pandas
df = pd.read_csv("data/airline_satisfaction.csv")
# Se elimina la primera columna del dataframe (era el número de fila) y el id
del df[df.columns[0]]
del df[df.columns[0]]
df.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [8]:
# Se obtienen estadísticas descriptivas de variables numéricas
print(df.describe())

                 Age  Flight Distance  Inflight wifi service  \
count  103904.000000    103904.000000          103904.000000   
mean       39.379706      1189.448375               2.729683   
std        15.114964       997.147281               1.327829   
min         7.000000        31.000000               0.000000   
25%        27.000000       414.000000               2.000000   
50%        40.000000       843.000000               3.000000   
75%        51.000000      1743.000000               4.000000   
max        85.000000      4983.000000               5.000000   

       Departure/Arrival time convenient  Ease of Online booking  \
count                      103904.000000           103904.000000   
mean                            3.060296                2.756901   
std                             1.525075                1.398929   
min                             0.000000                0.000000   
25%                             2.000000                2.000000   
50%            

In [9]:
# Se obtiene la distribución de los valores en la variable "satisfaction"
print(df["satisfaction"].value_counts())

neutral or dissatisfied    58879
satisfied                  45025
Name: satisfaction, dtype: int64


In [12]:
# Se calcula la correlación entre variables numéricas
print(df.corr())

                                        Age  Flight Distance  \
Age                                1.000000         0.099461   
Flight Distance                    0.099461         1.000000   
Inflight wifi service              0.017859         0.007131   
Departure/Arrival time convenient  0.038125        -0.020043   
Ease of Online booking             0.024842         0.065717   
Gate location                     -0.001330         0.004793   
Food and drink                     0.023000         0.056994   
Online boarding                    0.208939         0.214869   
Seat comfort                       0.160277         0.157333   
Inflight entertainment             0.076444         0.128740   
On-board service                   0.057594         0.109526   
Leg room service                   0.040583         0.133916   
Baggage handling                  -0.047529         0.063184   
Checkin service                    0.035482         0.073072   
Inflight service                  -0.049

# Modelo Dimensional Propuesto

### Tabla de Hechos (Satisfacción del Cliente)

* idCliente (clave foránea)
* idViaje (clave foránea)
* idServicio (clave foránea)
* IdTiempo (clave foránea)
* satisfaction
* DepartureDelay
* ArrivalDelay

### Dimensión Cliente

* IdCliente (clave primaria)
* Gender
* CustomerType
* age

### Dimensión Viaje

* idViaje (clave primaria)
* TypeTravel
* Class
* FlightDistance

### Dimensión Servicio

* idServicio (clave primaria)
* InflightWifiService
* DepartureArrivalTimeConvenient
* EaseOnlineBooking
* GateLocation
* FoodAndDrink
* OnlineBoarding
* SeatComfort
* InflightEntertainment
* OnboardService
* LegRoomService
* BaggageHandling
* CheckinService
* InflightService
* Cleanliness